In [1]:
import pandas as pd
import requests, os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def downloadCSV():
    n, m = 15, 16
    url = 'https://dati.istruzione.it/opendata'
    namef = '/ALUCORSOETASTA20{}{}20{}0831.csv'
    lsfile = {}
    for i in range(6):
        fname = namef.format(n, m, m)
        lsfile['./static/data'+fname] = 'OK'
        try:
            f = open('./static/data'+fname, 'x')
            f.close()
        except:
            pass
        if os.path.getsize('./static/data'+fname)==0:
            f = open('./static/data'+fname, 'wb')
            response = requests.get(url+fname)
            if response.reason == 'OK':
                f.write(response.content)
            lsfile['./static/data'+fname] = response.reason
            f.close()
        n += 1
        m += 1
    return lsfile

In [53]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [48]:
def getTablesResults(data):#, cod_mecc):
    tbls = []
    for i in [path for path, status in data.items() if status == 'OK']:
        tab = pd.read_csv(i)
        tbls.append(tab)
    tab = pd.concat(tbls, ignore_index=True)
    tab['ANNOSCOLASTICO'] = tab['ANNOSCOLASTICO'].apply(lambda n : str(n)[0:-2]+'/'+str(n)[-2:])
    tab = tab.groupby(['CODICESCUOLA', 'ANNOSCOLASTICO', 'ANNOCORSO']).agg({'ALUNNI': 'sum'})
    return tab

In [71]:
def getTableByCode(table, cod):
    df = tab.loc[cod].reset_index()
    df = df.pivot(index='ANNOCORSO', columns='ANNOSCOLASTICO', values='ALUNNI')
    df = pd.DataFrame(df.to_records())
    df.rename(columns={'ANNOCORSO':'Anno di corso'}, inplace=True)
    df = df.set_index('Anno di corso')
    df.to_csv('./static/results/{}.csv'.format(cod))
    return df;

In [76]:
tab = getTablesResults(downloadCSV())
print(tab)
sims = [c for c in list(set(tab.index.get_level_values(0))) if similar(c, 'MITF03801E')>0.75]
print(sims)
table = getTableByCode(tab, 'MITF03801E')
print(table)

                                       ALUNNI
CODICESCUOLA ANNOSCOLASTICO ANNOCORSO        
AGEE01102E   2015/16        1              53
                            2              62
                            3              57
                            4              45
                            5              61
...                                       ...
VVTL011519   2020/21        4              31
                            5              14
VVTN00201L   2020/21        1              18
                            2              34
                            3              27

[733335 rows x 1 columns]
['MITF00801P', 'MITF09601E', 'MOTF01801E', 'MITF03801E', 'MITD08101E', 'MITF08201L', 'MITF093013', 'MITF098016', 'MITD02301E', 'MITF063017']
               2015/16  2016/17  2017/18  2018/19  2019/20  2020/21
Anno di corso                                                      
1                  178      143      133      151      157      171
2                  144      14